## TRAINING
This notebook has as an objective to use a folder containing the formated .npy files to train a deep learning model that can be used in lieu of an FEA model to any accuracy above random, for a proof of concept that it is possible to do so, such that further research can be done afterward to optimize architecture, hyperparameters and data being fed in.

In [68]:
##  imports

import numpy as np
import torch
import torch.nn as nn
import pathlib
import PREPROCESSING_splitting as split

In [87]:
## first, create a class to load the files that are to be fed to the neural network, 
## for both the inputs and the outputs. To avoid confusion I'll refer to the inputs to the 
## FEA model as 'bound_conds' (boundary conditions), and the outputs as 'targets', while what 
## is fed into the neural network will be called an "input", and the output of the neural network "prediction"
## this function should probably be transformed into a dataloader later for a larger dataset, 
## but for now we'll keep it like this

def get_dataset(dataset_path, glob_parameter = '*.npy'):
    # concatenates all samples into a list of boundary conditions and a list of targets
    
    # set paths
    bound_cond_path = pathlib.Path(dataset_path, 'input')
    targets_path = pathlib.Path(dataset_path, 'output')

    test = pathlib.Path('D:/')
    
    
    
    
    # check if folder path is correct
    if bound_cond_path.is_dir() and targets_path.is_dir():
        print('path contains \'input\' and \'output\'')
        pass
    else:
        raise Exception (f'Argument dataset_path: {dataset_path} should contain folders ..\input and ..\output. Please check path')
    
    #create iterators for files
    bound_cond_iterator = bound_cond_path.glob(glob_parameter)
    targets_iterator = targets_path.glob(glob_parameter)
    
    #zip them to ensure that they are going through the same samples 
    samples_iterator = zip(bound_cond_iterator, targets_iterator)
    
    boundary_conditions = np.array([])
    targets = np.array([])
    
    for boundary_condition_files, targets_files in samples_iterator:
        if split.get_number(boundary_condition_files.name) == split.get_number(targets_files.name):
            
            boundary_conditions_temp = np.load(boundary_condition_files)
            targets_temp = np.load(targets_files)
            
            #start array if it hasn't been started yet
            if boundary_conditions.size == 0 and targets.size == 0:
                boundary_conditions = boundary_conditions_temp
                targets = targets_temp
            else:
                boundary_conditions = np.concatenate((boundary_conditions, boundary_conditions_temp), axis = 0)
                targets = np.concatenate((targets, targets_temp), axis = 0)
        else:
            raise Exception('the samples in the iterator are not synced')
    
    return torch.from_numpy(boundary_conditions).float(), torch.from_numpy(targets).float()
    

In [88]:
folders_path =  pathlib.Path('D:/Ansys Simulations/Project/2D/data/proof_of_concept/scaled/arrays')
dataset = get_dataset(dataset_path = folders_path)
print(dataset[0].shape)
print(dataset[1].shape)

path contains 'input' and 'output'
torch.Size([102, 7, 32, 32])
torch.Size([102, 4, 32, 32])


In [90]:
## define the neural network's general shape

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        ## convolutional layers
        self.conv1 = nn.Conv2d(in_channels = 7, out_channels = 14, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 14, out_channels = 16, kernel_size = 3, padding = 1)
        self.deconv1 = nn.ConvTranspose2d(in_channels = 16, out_channels = 14, kernel_size = 3, padding = 1)
        self.deconv2 = nn.ConvTranspose2d(in_channels = 14, out_channels = 10, kernel_size = 3, padding = 1)
        self.deconv3 = nn.ConvTranspose2d(in_channels = 10, out_channels = 7, kernel_size = 3, padding = 1)
        self.deconv4 = nn.ConvTranspose2d(in_channels = 7, out_channels = 4, kernel_size = 3, padding = 1)
        
        ## activation
        self.hardtanh = nn.Hardtanh()
        
        
        ##possible for later: MultiheadAttention
        
    def forward(self, boundary_conditions):
        print(boundary_conditions.shape)
        x = self.conv1(boundary_conditions)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.conv2(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv1(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv2(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv3(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv4(x)
        x = self.hardtanh(x)
        print(x.shape)
        return x
    
net = ConvNet().float()

## test to see if getting the correct size
net.forward(dataset[0][0:1,:,:,:])

torch.Size([1, 7, 32, 32])
torch.Size([1, 14, 32, 32])
torch.Size([1, 16, 32, 32])
torch.Size([1, 14, 32, 32])
torch.Size([1, 10, 32, 32])
torch.Size([1, 7, 32, 32])
torch.Size([1, 4, 32, 32])


tensor([[[[ 0.0563,  0.0505,  0.0692,  ...,  0.0724,  0.0842,  0.0798],
          [ 0.0645,  0.0497,  0.0631,  ...,  0.0630,  0.0623,  0.0594],
          [ 0.0647,  0.0423,  0.0532,  ...,  0.0513,  0.0518,  0.0399],
          ...,
          [ 0.0584,  0.0433,  0.0514,  ...,  0.0490,  0.0464,  0.0358],
          [ 0.0692,  0.0554,  0.0643,  ...,  0.0624,  0.0570,  0.0475],
          [ 0.0636,  0.0590,  0.0611,  ...,  0.0552,  0.0498,  0.0397]],

         [[ 0.1094,  0.0712,  0.0834,  ...,  0.0849,  0.0930,  0.0912],
          [ 0.1169,  0.0748,  0.0849,  ...,  0.0843,  0.0875,  0.0867],
          [ 0.1309,  0.0825,  0.0902,  ...,  0.0876,  0.0821,  0.0744],
          ...,
          [ 0.1380,  0.0717,  0.0836,  ...,  0.0937,  0.0868,  0.0778],
          [ 0.1240,  0.0726,  0.0830,  ...,  0.0977,  0.0904,  0.0858],
          [ 0.1104,  0.0538,  0.0558,  ...,  0.0730,  0.0604,  0.0698]],

         [[ 0.0116,  0.0229,  0.0184,  ...,  0.0200,  0.0108, -0.0122],
          [ 0.0854,  0.0631,  